In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e18/sample_submission.csv
/kaggle/input/playground-series-s3e18/train.csv
/kaggle/input/playground-series-s3e18/test.csv


In [2]:
data_train = pd.read_csv('/kaggle/input/playground-series-s3e18/train.csv')
data_test = pd.read_csv('/kaggle/input/playground-series-s3e18/test.csv')

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score,roc_curve , roc_auc_score,make_scorer
from sklearn.model_selection import cross_val_score,cross_validate

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

In [4]:
data_train.shape

(14838, 38)

In [5]:
data_train.describe()

,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,ExactMolWt,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,HallKierAlpha,HeavyAtomMolWt,Kappa3,MaxAbsEStateIndex,MinEStateIndex,NumHeteroatoms,PEOE_VSA10,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2,EC3,EC4,EC5,EC6
count,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000
mean,7418.500000,515.153604,9.135189,5.854307,6.738497,4.432570,5.253221,3.418749,1.773472,29.202823,10.435316,292.623087,1.236774,1.812070,2.255470,-1.207776,274.950211,5.874372,10.556443,-2.119772,8.584108,11.021644,17.790011,8.962440,11.318811,6.704487,15.666766,31.066423,13.636941,49.309959,0.458215,0.459226,0.667745,0.798962,0.313789,0.279081,0.144831,0.151570
std,4283.505982,542.456370,6.819989,4.647064,5.866444,3.760516,4.925065,3.436208,1.865898,31.728679,13.651843,225.384140,5.491284,5.495565,5.501200,0.935314,212.678755,45.730226,1.559331,2.066415,7.643769,13.958962,34.561655,19.756727,20.169745,10.865415,18.080208,33.896638,14.598554,29.174824,0.667948,0.668111,0.471038,0.400790,0.464047,0.448562,0.351942,0.358616
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.007276,-666.000000,-666.000000,-666.000000,-7.730000,0.000000,-104.040000,0.000000,-6.327514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.430556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3709.250000,149.103601,4.680739,2.844556,2.932842,1.949719,2.034468,1.160763,0.503897,5.969305,0.000000,148.037173,1.045455,1.690909,2.100000,-1.660000,136.109000,1.784008,9.926190,-4.659604,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.969305,6.420822,4.794537,30.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,7418.500000,290.987941,6.485270,4.052701,4.392859,2.970427,3.242775,1.948613,1.073261,17.353601,6.420822,206.042653,1.250000,1.865152,2.358491,-1.100000,194.276500,3.261011,10.421334,-1.265370,6.000000,6.041841,5.969305,0.000000,0.000000,0.000000,11.752550,20.075376,9.589074,41.666667,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,11127.750000,652.652585,11.170477,7.486791,8.527859,5.788793,6.609350,4.502070,2.534281,44.876559,12.841643,343.090331,1.500000,2.062153,2.500000,-0.570000,326.002000,5.848400,11.539743,-0.787037,10.000000,18.311899,15.645394,12.132734,13.847474,6.923737,17.721856,42.727765,14.912664,56.090650,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,14837.000000,4069.959780,69.551167,50.174588,53.431954,32.195368,34.579313,22.880836,16.072810,363.705954,99.936429,2237.318490,3.000000,3.200000,3.400000,0.820000,2035.133000,1512.242231,15.630251,6.000000,42.000000,97.663462,482.434223,375.425148,211.501279,100.348416,80.742293,492.729739,115.406157,384.450519,8.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
data_train.skew()

id                     0.000000
BertzCT                1.775677
Chi1                   1.679368
Chi1n                  1.722855
Chi1v                  1.782578
Chi2n                  1.667144
Chi2v                  1.748260
Chi3v                  1.644531
Chi4n                  1.893930
EState_VSA1            1.426010
EState_VSA2            1.914321
ExactMolWt             1.708846
FpDensityMorgan1    -120.930261
FpDensityMorgan2    -120.960846
FpDensityMorgan3    -120.830068
HallKierAlpha         -1.255378
HeavyAtomMolWt         1.704114
Kappa3                26.419611
MaxAbsEStateIndex     -1.698735
MinEStateIndex        -0.611405
NumHeteroatoms         1.564873
PEOE_VSA10             1.354507
PEOE_VSA14             4.529731
PEOE_VSA6              4.415531
PEOE_VSA7              3.104631
PEOE_VSA8              2.041539
SMR_VSA10              2.125224
SMR_VSA5               2.368562
SlogP_VSA3             1.613283
VSA_EState9            1.696281
fr_COO                 1.618629
fr_COO2 

In [7]:
Q1 = data_train['FpDensityMorgan1'].quantile(0.10)
Q2 = data_train['FpDensityMorgan1'].quantile(0.90)
print(Q1)
print(Q2)
print(Q1-(1.5*Q1))
print(Q2+(1.5*Q2))

0.909090909
1.75
-0.45454545449999995
4.375


In [8]:
data_train["FpDensityMorgan1"] = np.where(data_train["FpDensityMorgan1"] < -0.5, -0.5,data_train['FpDensityMorgan1'])
data_train["FpDensityMorgan1"] = np.where(data_train["FpDensityMorgan1"] > 4.64, 4.64,data_train['FpDensityMorgan1'])
data_train["FpDensityMorgan2"] = np.where(data_train["FpDensityMorgan2"] < -0.76, -0.76,data_train['FpDensityMorgan2'])
data_train["FpDensityMorgan2"] = np.where(data_train["FpDensityMorgan2"] > 5.83, 5.83,data_train['FpDensityMorgan2'])
data_train["FpDensityMorgan3"] = np.where(data_train["FpDensityMorgan3"] < -1.0, -1.0,data_train['FpDensityMorgan3'])
data_train["FpDensityMorgan3"] = np.where(data_train["FpDensityMorgan3"] > 7.14, 7.14,data_train['FpDensityMorgan3'])


In [9]:
data_test["FpDensityMorgan1"] = np.where(data_test["FpDensityMorgan1"] < -0.5, -0.5,data_test['FpDensityMorgan1'])
data_test["FpDensityMorgan1"] = np.where(data_test["FpDensityMorgan1"] > 4.64, 4.64,data_test['FpDensityMorgan1'])
data_test["FpDensityMorgan2"] = np.where(data_test["FpDensityMorgan2"] < -0.76, -0.76,data_test['FpDensityMorgan2'])
data_test["FpDensityMorgan2"] = np.where(data_test["FpDensityMorgan2"] > 5.83, 5.83,data_test['FpDensityMorgan2'])
data_test["FpDensityMorgan3"] = np.where(data_test["FpDensityMorgan3"] < -1.0, -1.0,data_test['FpDensityMorgan3'])
data_test["FpDensityMorgan3"] = np.where(data_test["FpDensityMorgan3"] > 7.14, 7.14,data_test['FpDensityMorgan3'])

In [10]:
y_train = data_train[['EC1','EC2']]
X_train = data_train.drop(['EC1','EC2','EC3','EC4','EC5','EC6','id'], axis= 1)
n_inputs, n_outputs = X_train.shape[1], y_train.shape[1]

In [11]:

X_test = data_test.drop(['id'], axis= 1)

In [12]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier

# initialize Label Powerset multi-label classifier
# with a gaussian naive bayes base classifier
lb_rf_classifier = LabelPowerset(RandomForestClassifier(n_estimators=50, criterion='gini',max_depth=5,min_samples_split=10, max_features="sqrt"))

# train
lb_rf_classifier.fit(X_train, y_train)

pred_prob1 = lb_rf_classifier.predict_proba(X_train).toarray()

train_auc_score1 = roc_auc_score(y_train['EC1'], pred_prob1[:,0])
train_auc_score2 = roc_auc_score(y_train['EC2'], pred_prob1[:,1])
print(train_auc_score1, train_auc_score2)

# predict
pred_prob2_lp = lb_rf_classifier.predict_proba(X_test).toarray()


# fpr1, tpr1, thresh1 = roc_curve(y_test['EC1'], pred_prob2[:,0], pos_label=1)
# fpr2, tpr2, thresh2 = roc_curve(y_test['EC2'], pred_prob2[:,1], pos_label=1)

# auc_score1 = roc_auc_score(y_test['EC1'], pred_prob2[:,0])
# auc_score2 = roc_auc_score(y_test['EC2'], pred_prob2[:,1])

# print(auc_score1)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.7242937868143513 0.625688899546467


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [13]:
# using Multi output classifier
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import RandomForestClassifier

# initialize Label Powerset multi-label classifier
# with a gaussian naive bayes base classifier
classifier_chain = ClassifierChain(RandomForestClassifier(n_estimators=50, criterion='gini',max_depth=5))

# train
classifier_chain.fit(X_train, y_train)

pred_prob1 = classifier_chain.predict_proba(X_train)
train_auc_score1 = roc_auc_score(y_train['EC1'], pred_prob1[:,0])
train_auc_score2 = roc_auc_score(y_train['EC2'], pred_prob1[:,1])
print(train_auc_score1, train_auc_score2)
# predict
pred_prob2_cf = classifier_chain.predict_proba(X_test)


# fpr1, tpr1, thresh1 = roc_curve(y_test['EC1'], pred_prob2[:,0], pos_label=1)
# fpr2, tpr2, thresh2 = roc_curve(y_test['EC2'], pred_prob2[:,1], pos_label=1)

# auc_score1 = roc_auc_score(y_test['EC1'], pred_prob2[:,0])
# auc_score2 = roc_auc_score(y_test['EC2'], pred_prob2[:,1])

# print(auc_score1)

0.7294914634515841 0.6309911797387502


In [14]:

result = pd.DataFrame(pred_prob2_cf)
result['id']= data_test['id']
result = result.rename(columns={0:'EC1',1:'EC2'})
cols = ['id','EC1','EC2']
submission_Data = result[cols]

In [15]:
submission_Data.to_csv('submission_Enzyme_cf.csv', encoding='utf-8', index=False)